
Feature scaling is a crucial step in the preprocessing pipeline for many 
statistical modeling and machine learning algorithms. Scalers transform features 
so that they are on the same scale, making it easier for the models to converge 
and perform optimally.

Importance:

* **Consistency** across features: without scaling , algorithms may become biased toward the larger-scale features

* **Faster convergence**: unscaled features can lead to slow convergence since large features values dominate the optimization steps

* **Model performance**: scaling is particularly critical for clustering algorithms, like K-means, and k-nearest neighbors, which depend on the distance between points. On the other hand, Tree-based algorithms are less sensitive to feature scaling. 

This notebook will implement some of the most important scalers.
The construction will be based on classes, each composed of the following functions:
1. __init__: 
2. fit
3. transform
4. inverse_transform



# MinMax Scaler

# Standard Scaler

# Robust Scaler